# 六边形单元格面积的变化


&emsp;&emsp;六边形单元格的大小根据它们在球体上的位置而变化。这种扭曲来自于单元格投影到球体上的等距投影。

&emsp;&emsp;这个笔记本计算了每个分辨率下球体上**六边形**单元格面积的最小值和最大值。我们排除了五边形，因为它们是特殊情况。

## 穷举法面积计算

&emsp;&emsp;我们可以通过在每个层次下对所有的**六边形**（排除五边形）进行穷举搜索来找到最小和最大面积的六边形。然而，对于更高层次的格网，这种搜索变得不切实际地慢。

In [1]:
# 导入 h3 和 tabulate 库
import h3
from tabulate import tabulate

# 定义一个函数，用于找出给定层次下面积最大或最小的六边形
def brute_extreme_area_hex(res, min_or_max=min):
    """
    注意：由于对称性，面积最大或最小的六边形不是唯一的。
    """
    # 获取层次为0的所有单元
    cells = h3.get_res0_indexes()
    # 将层次为0的所有单元扩展到给定层次
    cells = h3.uncompact(cells, res)
    # 从扩展后的单元中过滤出所有六边形（排除五边形）
    cells = (c for c in cells if not h3.h3_is_pentagon(c))
    
    # 找出面积最小或最大的六边形
    h = min_or_max(cells, key=h3.cell_area)
    
    return h


In [2]:
# 设置层次为4
res = 4

# 找出该层次下面积最大的六边形
h_max = brute_extreme_area_hex(res, max)
# 找出该层次下面积最小的六边形
h_min = brute_extreme_area_hex(res, min)

# 计算在该层次下，面积最大和最小的六边形的面积比
h3.cell_area(h_max)/h3.cell_area(h_min)


1.970457453618376

## 使用递归搜索

&emsp;&emsp;为了找到每个分辨率下面积最小/最大的六边形，我们可以使用一种递归过程，其中我们在粗糙的分辨率下找到极值六边形，然后在更细的分辨率下查看那个六边形（确切地说，是它的子元格）周围的区域。由于面积扭曲的连续性质，这种方法应该能给我们提供精确的结果。

In [3]:
# 定义一个函数，找出给定六边形的 k-ring 中面积最小或最大的六边形
def extreme_small_neighborhood(h, min_or_max=min, k=1):
    """取 h 所在的 k-ring 中的所有单元，
    对每个单元进行下一层次的扩展，然后找出面积最小或最大的六边形。
    """
    # 获取 h 所在的 k-ring
    cells = h3.k_ring(h, k)
    
    # 对每个单元进行下一层次的扩展
    res =  h3.h3_get_resolution(h) + 1
    cells = h3.uncompact(cells, res)
    
    # 过滤出六边形
    cells = (c for c in cells if not h3.h3_is_pentagon(c))

    # 找出面积最小或最大的六边形
    h = min_or_max(cells, key=h3.cell_area)
        
    return h

# 定义一个函数，找出在给定层次下面积最小或最大的六边形
def extreme_area_hex(res, min_or_max=min, k=1):
    """通过递归地搜索每个单元的小邻域，找出在给定层次下面积最小或最大的六边形。
    """
    if res == 0:
        # 获取层次0的所有单元
        cells = h3.get_res0_indexes()
        # 过滤出六边形
        cells = (c for c in cells if not h3.h3_is_pentagon(c))
        # 找出面积最小或最大的六边形
        return min_or_max(cells, key=h3.cell_area)
    else:
        # 先找出在上一层次（res-1）下的面积最小或最大的六边形
        h = extreme_area_hex(res-1, min_or_max)
        # 然后找出该六边形的下一层次的 k-ring 中面积最小或最大的六边形
        h = extreme_small_neighborhood(h, min_or_max, k=k)
        return h


## 比较遍历搜索和递归搜索

In [4]:
res = 4

h_max = extreme_area_hex(res, max)
h_min = extreme_area_hex(res, min)

h3.cell_area(h_max)/h3.cell_area(h_min)

1.9704574536181565

In [5]:
res = 4

h_max = brute_extreme_area_hex(res, max)
h_min = brute_extreme_area_hex(res, min)

h3.cell_area(h_max)/h3.cell_area(h_min)

1.970457453618376

## 格式化展示

In [6]:
# 定义一个函数，计算每个层次下六边形的最大面积、最小面积和它们的比例
def stats():
    """
    对于每个层次，返回以下值：
    - 层次
    - 六边形的最大面积
    - 六边形的最小面积
    - 最大面积和最小面积的比例
    """
    # 遍历每个层次
    for res in range(16):
        # 找出面积最大的六边形
        h_max = extreme_area_hex(res, max)
        # 找出面积最小的六边形
        h_min = extreme_area_hex(res, min)
        
        # 计算面积最大的六边形的面积
        a_max = h3.cell_area(h_max)
        # 计算面积最小的六边形的面积
        a_min = h3.cell_area(h_min)

        # 返回层次、最大面积、最小面积和它们的比例
        yield res, a_min, a_max, a_max/a_min


In [8]:
list(stats())

[(0, 4106166.3344639097, 4977807.027442013, 1.2122760312124332),
 (1, 447684.2018179404, 729486.8752753432, 1.629467540541007),
 (2, 56786.62288947406, 104599.80721892552, 1.8419797110053904),
 (3, 7725.5057696395, 14950.773301378897, 1.9352484804469383),
 (4, 1084.0056353627895, 2135.986983964694, 1.9704574536181565),
 (5, 153.76624444796315, 305.1443087785757, 1.9844687621400625),
 (6, 21.910021012645327, 43.592111685004156, 1.989596982122703),
 (7, 3.1268360301058844, 6.2274459054874844, 1.9916125583587456),
 (8, 0.44652617408406686, 0.8896351574995826, 1.9923471660411383),
 (9, 0.06378022693686203, 0.127090737360365, 1.9926353897451001),
 (10, 0.009110980969669582, 0.018155819634707197, 1.9927403750647539),
 (11, 0.0013015418135795185, 0.002593688519475598, 1.9927815552405495),
 (12, 0.00018593314532466473, 0.0003705269313802326, 1.992796554553207),
 (13, 2.6561799507940082e-05, 5.293241878201183e-05, 1.9928024366793677),
 (14, 3.7945387017232687e-06, 7.5617741133636225e-06, 1.9928

In [9]:
res_fmt = '{:2d}'
float_fmt = '{:20,.9f}'
ratio_fmt = '{:.6f}'

fmt = f'{res_fmt} {float_fmt} {float_fmt}  {ratio_fmt}'

for res, a_min, a_max, ratio in stats():
    print(fmt.format(res, a_max, a_min, ratio))

 0  4,977,807.027442013  4,106,166.334463910  1.212276
 1    729,486.875275343    447,684.201817940  1.629468
 2    104,599.807218926     56,786.622889474  1.841980
 3     14,950.773301379      7,725.505769639  1.935248
 4      2,135.986983965      1,084.005635363  1.970457
 5        305.144308779        153.766244448  1.984469
 6         43.592111685         21.910021013  1.989597
 7          6.227445905          3.126836030  1.991613
 8          0.889635157          0.446526174  1.992347
 9          0.127090737          0.063780227  1.992635
10          0.018155820          0.009110981  1.992740
11          0.002593689          0.001301542  1.992782
12          0.000370527          0.000185933  1.992797
13          0.000052932          0.000026562  1.992802
14          0.000007562          0.000003795  1.992805
15          0.000001080          0.000000542  1.992805


In [10]:
def fmt_float(x):
    s = float_fmt
    return s.format(x)

def fmt_ratio(x):
    s = ratio_fmt
    return s.format(x)

fmt_stats = [
    (a, fmt_float(b), fmt_float(c), fmt_ratio(d))
    for a,b,c,d in stats()
]

fmt_stats

[(0, ' 4,106,166.334463910', ' 4,977,807.027442013', '1.212276'),
 (1, '   447,684.201817940', '   729,486.875275343', '1.629468'),
 (2, '    56,786.622889474', '   104,599.807218926', '1.841980'),
 (3, '     7,725.505769639', '    14,950.773301379', '1.935248'),
 (4, '     1,084.005635363', '     2,135.986983965', '1.970457'),
 (5, '       153.766244448', '       305.144308779', '1.984469'),
 (6, '        21.910021013', '        43.592111685', '1.989597'),
 (7, '         3.126836030', '         6.227445905', '1.991613'),
 (8, '         0.446526174', '         0.889635157', '1.992347'),
 (9, '         0.063780227', '         0.127090737', '1.992635'),
 (10, '         0.009110981', '         0.018155820', '1.992740'),
 (11, '         0.001301542', '         0.002593689', '1.992782'),
 (12, '         0.000185933', '         0.000370527', '1.992797'),
 (13, '         0.000026562', '         0.000052932', '1.992802'),
 (14, '         0.000003795', '         0.000007562', '1.992805'),
 (15,

In [12]:
headers = [
    '层次',
    '最小的六边形格网面积(km^2)',
    '最大的六边形格网面积(km^2)',
    '比率 (max/min)'
]
out = tabulate(fmt_stats, headers=headers, tablefmt='pipe', stralign='right', disable_numparse=True)

print(out)

|   层次 |   最小的六边形格网面积(km^2) |   最大的六边形格网面积(km^2) |   比率 (max/min) |
|-------:|-----------------------------:|-----------------------------:|-----------------:|
|      0 |          4,106,166.334463910 |          4,977,807.027442013 |         1.212276 |
|      1 |            447,684.201817940 |            729,486.875275343 |         1.629468 |
|      2 |             56,786.622889474 |            104,599.807218926 |         1.841980 |
|      3 |              7,725.505769639 |             14,950.773301379 |         1.935248 |
|      4 |              1,084.005635363 |              2,135.986983965 |         1.970457 |
|      5 |                153.766244448 |                305.144308779 |         1.984469 |
|      6 |                 21.910021013 |                 43.592111685 |         1.989597 |
|      7 |                  3.126836030 |                  6.227445905 |         1.991613 |
|      8 |                  0.446526174 |                  0.889635157 |         1.992347 |
|      9 |  

In [13]:
from IPython.display import Markdown
Markdown(out)

|   层次 |   最小的六边形格网面积(km^2) |   最大的六边形格网面积(km^2) |   比率 (max/min) |
|-------:|-----------------------------:|-----------------------------:|-----------------:|
|      0 |          4,106,166.334463910 |          4,977,807.027442013 |         1.212276 |
|      1 |            447,684.201817940 |            729,486.875275343 |         1.629468 |
|      2 |             56,786.622889474 |            104,599.807218926 |         1.841980 |
|      3 |              7,725.505769639 |             14,950.773301379 |         1.935248 |
|      4 |              1,084.005635363 |              2,135.986983965 |         1.970457 |
|      5 |                153.766244448 |                305.144308779 |         1.984469 |
|      6 |                 21.910021013 |                 43.592111685 |         1.989597 |
|      7 |                  3.126836030 |                  6.227445905 |         1.991613 |
|      8 |                  0.446526174 |                  0.889635157 |         1.992347 |
|      9 |                  0.063780227 |                  0.127090737 |         1.992635 |
|     10 |                  0.009110981 |                  0.018155820 |         1.992740 |
|     11 |                  0.001301542 |                  0.002593689 |         1.992782 |
|     12 |                  0.000185933 |                  0.000370527 |         1.992797 |
|     13 |                  0.000026562 |                  0.000052932 |         1.992802 |
|     14 |                  0.000003795 |                  0.000007562 |         1.992805 |
|     15 |                  0.000000542 |                  0.000001080 |         1.992805 |